## OPENAI TOKEN

In [16]:
%env OPENAI_API_KEY=<OPENAI TOKEN>

env: OPENAI_API_KEY=<OPENAI TOKEN>


## Set DB configuration

In [2]:
DATABASE = "postgres"
USER = "postgres"
PASSWORD = "pgpassword"
HOST = "localhost"
PORT = "5432"

DATABASE_URL = (
    f"postgresql+psycopg2://{USER}:{PASSWORD}@{HOST}:{PORT}/{DATABASE}"
)

## Configure the SQL Agent

In [8]:
from langchain import SQLDatabase
from langchain.agents import initialize_agent
from langchain.agents import AgentType
from langchain.chat_models import ChatOpenAI
from langchain.agents.agent_toolkits import SQLDatabaseToolkit
from langchain.schema import SystemMessage


db = SQLDatabase.from_uri(DATABASE_URL)
toolkit = SQLDatabaseToolkit(llm=ChatOpenAI(temperature=0), db=db)

agent_kwargs = {
    "system_message": SystemMessage(content="You are an expert SQL data analyst.")
}

llm = ChatOpenAI(temperature=0, model="gpt-3.5-turbo")
agent = initialize_agent(
    toolkit.get_tools(),
    llm,
    agent=AgentType.OPENAI_FUNCTIONS,
    verbose=True,
    agent_kwargs=agent_kwargs,
)

## Testing

In [9]:
agent.run("Describe the table transactions")



> Entering new AgentExecutor chain...

Invoking: `sql_db_schema` with `transactions`



CREATE TABLE transactions (
	date TEXT, 
	supplier TEXT, 
	description TEXT, 
	value BIGINT, 
	classification TEXT
)

/*
3 rows from transactions table:
date	supplier	description	value	classification
15/08/2016	Creative Video Productions Ltd	Kelvin Hall	26866	Other
29/05/2017	John Graham Construction Ltd	Causewayside Refurbishment	74806	Building Improvement
29/05/2017	Morris & Spottiswood Ltd	George IV Bridge Work	56448	Building Improvement
*/The table "transactions" has the following columns:

1. date: TEXT
2. supplier: TEXT
3. description: TEXT
4. value: BIGINT
5. classification: TEXT

Here are three sample rows from the "transactions" table:

1. date: 15/08/2016, supplier: Creative Video Productions Ltd, description: Kelvin Hall, value: 26866, classification: Other
2. date: 29/05/2017, supplier: John Graham Construction Ltd, description: Causewayside Refurbishment, value: 74806, classification:

'The table "transactions" has the following columns:\n\n1. date: TEXT\n2. supplier: TEXT\n3. description: TEXT\n4. value: BIGINT\n5. classification: TEXT\n\nHere are three sample rows from the "transactions" table:\n\n1. date: 15/08/2016, supplier: Creative Video Productions Ltd, description: Kelvin Hall, value: 26866, classification: Other\n2. date: 29/05/2017, supplier: John Graham Construction Ltd, description: Causewayside Refurbishment, value: 74806, classification: Building Improvement\n3. date: 29/05/2017, supplier: Morris & Spottiswood Ltd, description: George IV Bridge Work, value: 56448, classification: Building Improvement'

In [11]:
agent.run("what is the highest value of a transaction?")



> Entering new AgentExecutor chain...

Invoking: `sql_db_query` with `SELECT MAX(amount) AS highest_transaction FROM transactions`


Error: (psycopg2.errors.UndefinedColumn) column "amount" does not exist
LINE 1: SELECT MAX(amount) AS highest_transaction FROM transactions
                   ^

[SQL: SELECT MAX(amount) AS highest_transaction FROM transactions]
(Background on this error at: https://sqlalche.me/e/20/f405)
Invoking: `sql_db_schema` with `transactions`



CREATE TABLE transactions (
	date TEXT, 
	supplier TEXT, 
	description TEXT, 
	value BIGINT, 
	classification TEXT
)

/*
3 rows from transactions table:
date	supplier	description	value	classification
15/08/2016	Creative Video Productions Ltd	Kelvin Hall	26866	Other
29/05/2017	John Graham Construction Ltd	Causewayside Refurbishment	74806	Building Improvement
29/05/2017	Morris & Spottiswood Ltd	George IV Bridge Work	56448	Building Improvement
*/
Invoking: `sql_db_query` with `SELECT MAX(value) AS highest_transaction FROM t

'The highest value of a transaction is 3,422,500.'

In [15]:
agent.run("Could you give me the statistics of the transaction's values?")



> Entering new AgentExecutor chain...

Invoking: `sql_db_query` with `{'query': 'SELECT COUNT(*) AS total_transactions, SUM(value) AS total_value, AVG(value) AS average_value, MIN(value) AS min_value, MAX(value) AS max_value FROM transactions'}`


[(101, Decimal('13877908'), Decimal('137405.029702970297'), 26506, 3422500)]The statistics of the transaction's values are as follows:

- Total number of transactions: 101
- Total value of transactions: $13,877,908
- Average value of transactions: $137,405.03
- Minimum value of a transaction: $26,506
- Maximum value of a transaction: $3,422,500

> Finished chain.


"The statistics of the transaction's values are as follows:\n\n- Total number of transactions: 101\n- Total value of transactions: $13,877,908\n- Average value of transactions: $137,405.03\n- Minimum value of a transaction: $26,506\n- Maximum value of a transaction: $3,422,500"